In [1]:
import pandas as pd
import pyarrow.parquet as pq
import os
import airportsdata 
from geopy.distance import geodesic

import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import pycountry
import pycountry_convert as pc
import plotly.io as pio

import folium
from folium.plugins import FastMarkerCluster
from folium.plugins import HeatMap

# from math import radians, sin, cos, sqrt, atan2
# import heapq

# 1. Data preparation

In [117]:
# Load airport data to get latitude and longitude
airports = airportsdata.load()

# Load provided air data from 2021
dataset = pq.ParquetDataset('c:\\Users\\lukas\\OneDrive - Imperial College London\\0_Sustainable Transport\\2021_COP26',
                            ) # filters=[('columnName', 'in', filterList)]
df = dataset.read(use_threads=True).to_pandas()

In [118]:
# Exclude private jets
private_jet = ['CL60','P180','GLF5','GLEX','GLF6','F2TH','C17','LJ60','F900','GLF4','E35L','CL35','C30J',
               'GL5T','ASTR','C130','BE20','H25B','L410','F100','CL30','LJ35','PC24','C56X','E55P','PAY3']
df = df[~df['aircraft_type_icao'].isin(private_jet)]

In [119]:
# Select flights to dubai
df_dubai = df[df['destination_airport']== 'OMDB']

# Group by 'origin_airport' and select the first occurrence of each group
unique_origins = df_dubai.groupby('origin_airport').first().reset_index()
unique_origins = unique_origins[['origin_airport','longitude_first_waypoint','latitude_first_waypoint']]

file_path = os.path.join('output', 'unique_destinations_globally.xlsx')
unique_origins.to_excel(file_path, index=False)


In [127]:
# Load data with delegates
delegates_path = os.getcwd() + '/data/origin_airports_delegates.csv'
data_delegates = pd.read_csv(delegates_path)

iata_icao_path = os.getcwd() + '/data/iata-icao.csv'
data_iata_icao = pd.read_csv(iata_icao_path)

In [128]:
airport_location = data_delegates.merge(data_iata_icao, on='iata', how='left')[['iata','icao','total_delegates_GDP','country_code','airport','latitude','longitude']]

# Check for NAN values 
# airport_location.isna()
# airport_location = airport_location.dropna()#isna().sum()#.any(axis=1).count
# airport_location[airport_location.isna().any(axis=1)]
airport_location["country_code"][77] =  "NA"

# 2. Calculation Distance to Dubai

In [129]:
dubai_airport = pd.DataFrame({'latitude_A': [airports.get('OMDB')['lat']],
                     'longitude_A': [airports.get('OMDB')['lon']]})

delegate_airports = airport_location[['latitude','longitude']]

# Function to calculate distance using geopy.distance
def calculate_distance(row):
    coord_A = (dubai_airport['latitude_A'].iloc[0], dubai_airport['longitude_A'].iloc[0])
    coord_B = (row['latitude'], row['longitude'])
    return geodesic(coord_A, coord_B).kilometers

airport_location['distance_to_dubai'] = delegate_airports.apply(calculate_distance,axis=1)
# airport_location[airport_location['distance_to_dubai'] >= 10000]
airport_location.iloc[:3]

,iata,icao,total_delegates_GDP,country_code,airport,latitude,longitude,distance_to_dubai
0,AUH,OMAA,4557,AE,Abu Dhabi International Airport,24.4330,54.6511,115.950398
1,GRU,SBGR,3643,BR,Sao Paulo/Guarulhos International Airport,-23.4262,-46.4800,12218.280093
2,PEK,ZBAA,6672,CN,Beijing Capital International Airport,40.0725,116.5980,5856.254969


In [130]:
print(airport_location[airport_location['distance_to_dubai'] >= 10000]['distance_to_dubai'].count(),"flights with over 10,000 km")

55 flights with over 10,000 km


In [135]:
print("Total km flown to COP28:",round(sum(airport_location['distance_to_dubai']),0))
print("Total Delegates:          ",sum(airport_location.total_delegates_GDP))

Total km flown to COP28: 1401384.0
Total Delegates:           81708


# 3. Analysis 
## Participants from each Country

In [136]:
def country_code_to_name(country_code):
    try:
        return pycountry.countries.get(alpha_2=country_code).name
    except AttributeError:
        return None

plot_data = airport_location[:][:40]
plot_data
plot_data['country_name'] = plot_data['country_code'].apply(country_code_to_name)

In [137]:
# Rename due to length of name
plot_data.loc[0,"country_name"] = 'UAE'
plot_data.loc[12,"country_name"] = 'Tanzania'
plot_data.loc[17,"country_name"] = 'Korea'
plot_data.loc[20,"country_name"] = 'Congo'
plot_data.loc[21,"country_name"] = 'Russia'

In [140]:
sorted_indices = sorted(range(len(plot_data['total_delegates_GDP'])), key=lambda i: plot_data['total_delegates_GDP'][i])
sorted_categories = [plot_data['country_name'][i] for i in sorted_indices]
sorted_values = [plot_data['total_delegates_GDP'][i] for i in sorted_indices]

fig = go.Figure(data=[go.Bar(
    y=sorted_categories,  # y-axis categories
    x=sorted_values,      # x-axis values
    orientation='h'  # horizontal orientation
)])

# Add title and labels
fig.update_layout(
    title='Participants from each country (Top 40)',
    yaxis=dict(title='Countries'),
    xaxis=dict(title='# participants'),
    width=400,
    height=800
)

# Show the chart
#fig.write_image("participants_country.png")
fig.show()
# file_path = os.path.join('output', 'participants_country_GDP.png')
# non_direct.to_excel(file_path, index=False)

pio.write_image(fig, 'participants_country_GDP.png')


# 4. Find Flight Connections from Origin to Dubai
## 4.1 Direct Flight

In [107]:
destination_dubai = df[df['destination_airport'] == 'OMDB']
# destination_dubai[destination_dubai['origin_airport'] == 'DNMM'] #[['origin_airport_name','destination_airport_name']]
# destination_dubai['origin_airport'] = destination_dubai['origin_airport'].astype('str')#.dtypes
print(len(destination_dubai))
destination_dubai.iloc[:3]


23328


,callsign,icao_address,flight_number,tail_number,aircraft_type_icao,aircraft_engine_type,origin_airport,origin_airport_name,origin_country,destination_airport,...,so2_kg,sulphates_kg,nvpm_data_source,nvpm_mass_kg,nvpm_number,nvpm_ei_n_mean,sdr_mean,night_flight,night_segments,pct_night_dist
93,FDB1916,8963B5,FZ1916,A6-FES,B738,Jet,URML,Makhachkala Uytash International Airport,RU,OMDB,...,8.002414,0.163315,ICAO EDB,0.363317,7.447394e+18,1.116772e+15,14.200313,False,True,74.495262
213,FDB1462,8963B1,FZ1462,A6-FEO,B738,Jet,UCFM,Manas International Airport,KG,OMDB,...,11.051537,0.225542,ICAO EDB,0.491998,1.079065e+19,1.171672e+15,90.088696,False,True,23.943959
1194,FDB448,896331,FZ448,A6-FDZ,B738,Jet,VOMM,Chennai International Airport,IN,OMDB,...,10.664273,0.217638,ICAO EDB,0.453730,9.797427e+18,1.102458e+15,122.496717,False,False,0.000000


In [108]:
# # Alternative 
# def inner_merge_on_columns(df1, df2, column1, column2):
#     merged_data = []
#     for value1 in df1[column1]:
#         for value2 in df2[column2]:
#             if value1 == value2:
#                 row1 = df1[df1[column1] == value1].iloc[0].to_dict()
#                 row2 = df2[df2[column2] == value2].iloc[0].to_dict()
#                 merged_data.append({**row1, **row2})
#     return pd.DataFrame(merged_data)

# merged_df = inner_merge_on_columns(airport_location,destination_dubai,"icao","origin_airport")
# merged_df['icao'].unique()


# Merging airport origin with the existing flights from 2021
merged_df = airport_location.merge(destination_dubai, left_on='icao', right_on='origin_airport', how='inner')
test = merged_df['icao'].unique()
print("Flights who have a direct flight to Dubai:",test.size)

Flights who have a direct flight to Dubai: 87


### Select unique flights from origin to destination airport

In [109]:
merged_df = merged_df.sort_values(by=['icao', 'co2_kg'])
unique_flights_df = merged_df.drop_duplicates(subset=['icao'])
unique_flights_df.reset_index(inplace=True)

# Relevant columns
unique_flights_df = unique_flights_df[['icao', 'total_delegates', 'country_code', 'origin_airport','destination_airport',
       'distance_to_dubai', 'assumed_load_factor', 'aircraft_type_icao', 'total_fuel_burn','co2_kg', 
       'nox_kg','hc_kg', 'oc_kg', 'h2o_kg', 'so2_kg', 'sulphates_kg', 'latitude','longitude'
       ]]

file_path = os.path.join('output', 'flights_without_transit.xlsx')
unique_flights_df.to_excel(file_path, index=False)
unique_flights_df.iloc[:3]

,icao,total_delegates,country_code,origin_airport,destination_airport,distance_to_dubai,assumed_load_factor,aircraft_type_icao,total_fuel_burn,co2_kg,nox_kg,hc_kg,oc_kg,h2o_kg,so2_kg,sulphates_kg,latitude,longitude
0,DAAG,33,DZ,DAAG,OMDB,5076.841504,0.625348,A332,28026.934111,88537.084858,377.908845,2.433629,0.560539,34473.128957,33.632321,0.686374,36.69100,3.215410
1,DGAA,618,GH,DGAA,OMDB,6292.961752,0.623285,B737,16668.516582,52655.843883,181.799396,1.341575,0.333370,20502.275396,20.002220,0.408209,5.60519,-0.166786
2,DTTA,221,TN,DTTA,OMDB,4451.568920,0.619923,B77L,30416.465762,96085.615343,554.819782,6.042459,0.608329,37412.252888,36.499759,0.744893,36.85100,10.227200


In [110]:
unique_flights_df['aircraft_type_icao'].unique()

array(['A332', 'B737', 'B77L', 'B38M', 'A319', 'B738', 'A20N', 'B77W',
       'BCS3', 'B733', 'B734', 'B39M', 'A333', 'A388', 'B789', 'FA7X',
       'CRJ2', 'A19N', 'A320', 'E75S', 'A21N', 'DH8D', 'G280', 'A359'],
      dtype=object)

### Export flights without direct connection

In [111]:
# All origins who 
airport_location['existing_flight'] = airport_location['icao'].isin(merged_df['origin_airport'])
# airport_location[airport_location['existing_flight'] == False]

non_direct = airport_location[airport_location['existing_flight'] == False]
# test = pd.merge(export, data_iata_icao, on='icao', how='left')
# test.drop_duplicates(subset=['icao'])



file_path = os.path.join('output', 'non_existing_flights.xlsx')
non_direct.to_excel(file_path, index=False)

non_direct.reset_index(inplace=True)
non_direct = non_direct[['iata','icao','total_delegates','country_code','airport','latitude','longitude','distance_to_dubai','existing_flight']]
non_direct.iloc[:3]

,iata,icao,total_delegates,country_code,airport,latitude,longitude,distance_to_dubai,existing_flight
0,LOS,DNMM,1411,NG,Murtala Muhammed International Airport,6.57737,3.32116,5899.623220,False
1,RBA,GMME,823,MA,Rabat-Sale Airport,34.05150,-6.75152,6003.076078,False
2,YOW,CYOW,1830,CA,Ottawa Macdonald-Cartier International Airport,45.32250,-75.66920,10756.579780,False


## 4.2 Non-direct Flight (hub-airport required)

In [112]:
def is_defined(obj):
    if obj is not None:
        return True
    else:
        return False
    
all_flights_non_direct = None

path_transit = os.getcwd() + '/data/transit_flight_stopover.csv'
df_stopover = pd.read_csv(path_transit)
print(len(df_stopover))
df_stopover.iloc[:3]


114


,iata,icao,total_delegates,country_code,latitude,longitude,closest,second_c,third_c,fourth_c,fifth_c,sixth_c,seventh_c,eigth_c,nineth_c,tenth_c,to_dubai
0,LOS,DNMM,1411,NG,6.57737,3.32116,DGAA,FNLU,GOBD,HRYR,HJJJ,GMMN,HUEN,DAAG,DTTA,LICC,OMDB
1,RBA,GMMN,823,MA,34.05150,-6.75152,GMMN,LPPT,LEMD,LEZG,DAAG,LEBL,LFBO,LFLL,LFMN,DTTA,OMDB
2,YOW,CYOW,1830,CA,45.32250,-75.66920,CYYZ,KEWR,KJFK,KPHL,KBOS,KIAD,KBGR,KLCK,KSDF,KORD,OMDB


In [113]:
df_stopover.iloc[:,4:14].columns

Index(['latitude', 'longitude', 'closest', 'second_c', 'third_c', 'fourth_c',
       'fifth_c', 'sixth_c', 'seventh_c', 'eigth_c'],
      dtype='object')

In [114]:
print('Flights without direct flight to dubai (hub_airport required): ',len(df_stopover), '\n')

for i_closeness in df_stopover.iloc[:,6:16].columns:
    print('-----------------------------------------------------------\n')
    merged_df = pd.merge(df_stopover, df, left_on=['icao', i_closeness], right_on=['origin_airport', 'destination_airport'], how='inner')
    merged_df = merged_df.drop_duplicates(subset=['icao'], keep='first')
    merged_df.reset_index(inplace=True)
    merged_df = merged_df.rename(columns={'destination_airport':'hub_airport'})
    merged_df = merged_df[['origin_airport','hub_airport',
        'assumed_load_factor', 'aircraft_type_icao', 'total_fuel_burn','co2_kg', 
        'nox_kg','hc_kg', 'oc_kg', 'h2o_kg', 'so2_kg', 'sulphates_kg','to_dubai'
        ]]
    #print(len(merged_df))
    merged_df.iloc[:3]
    print(i_closeness,'\n Flights found: ', len(merged_df))
    #############################
    ##### Hub --> Dubai
    #############################
    # Remaining flights to be found! 
    occurrences_count = merged_df['origin_airport'].value_counts().to_dict()

    # Delete the same number of occurrences from df_stopover['icao'] for each value in final_merged_df['origin_airport']
    for value, count in occurrences_count.items():
        df_stopover = df_stopover.drop(df_stopover[df_stopover['icao'] == value].head(count).index)

    len(df_stopover)

    results_closest = merged_df[['hub_airport','to_dubai']]
    destination_dubai = pd.merge(results_closest, df, left_on=['hub_airport', 'to_dubai'], right_on=['origin_airport', 'destination_airport'], how='inner')
    destination_dubai = destination_dubai.drop_duplicates(subset=['hub_airport'], keep='first')
    destination_dubai.reset_index(inplace=True)
    #print(len(destination_dubai))
    destination_dubai = destination_dubai[['origin_airport','hub_airport',
        'assumed_load_factor', 'aircraft_type_icao', 'total_fuel_burn','co2_kg', 
        'nox_kg','hc_kg', 'oc_kg', 'h2o_kg', 'so2_kg', 'sulphates_kg','to_dubai'
        ]]
    destination_dubai.iloc[:3]

    combined_dfs = []
    
    # Iterate over unique hub_airport values
    for hub_airport in merged_df['hub_airport'].unique():
        # Filter the row from destination_dubai
        row = destination_dubai[destination_dubai['origin_airport'] == hub_airport].iloc[0]

        # Number of times to add the row
        num_columns_to_add = (merged_df['hub_airport'] == hub_airport).sum()

        # Create a list of copies of row
        copies_of_row = [row] * num_columns_to_add

        # Concatenate the copies along the columns axis
        combined_df = pd.concat(copies_of_row, axis=1)

        # Transpose the DataFrame to have the rows and columns swapped
        combined_df = combined_df.T

        # Append the combined_df to the list
        combined_dfs.append(combined_df)

    if is_defined(all_flights_non_direct):
        all_flights_non_direct = pd.concat([all_flights_non_direct] + [merged_df] + combined_dfs, ignore_index=True)
    else:
        all_flights_non_direct = pd.concat([merged_df] + combined_dfs, ignore_index=True)
    
    print(' Total flights found: ', len(all_flights_non_direct))
    print(' Remaining flights: ',len(df_stopover),' \n')
    

# all_flights_non_direct.iloc[:3]

Flights without direct flight to dubai (hub_airport required):  114 

-----------------------------------------------------------

closest 
 Flights found:  56


 Total flights found:  112
 Remaining flights:  58  

-----------------------------------------------------------

second_c 
 Flights found:  9
 Total flights found:  130
 Remaining flights:  49  

-----------------------------------------------------------

third_c 
 Flights found:  9
 Total flights found:  148
 Remaining flights:  40  

-----------------------------------------------------------

fourth_c 
 Flights found:  2
 Total flights found:  152
 Remaining flights:  38  

-----------------------------------------------------------

fifth_c 
 Flights found:  3
 Total flights found:  158
 Remaining flights:  35  

-----------------------------------------------------------

sixth_c 
 Flights found:  1
 Total flights found:  160
 Remaining flights:  34  

-----------------------------------------------------------

seventh_c 
 Flights found:  2
 Total flights found:  164
 Remaining flights:  32  

-----------------------------------------------------------

eigth_c 
 Flights found

In [115]:
file_path = os.path.join('output', 'flights_non_direct_missing30.xlsx')
all_flights_non_direct.to_excel(file_path, index=False)

In [116]:
all_flights_non_direct['aircraft_type_icao'].unique()

array(['A332', 'E190', 'A320', 'A333', 'B350', 'B77W', 'B788', 'E75L',
       'B737', 'B738', 'A359', 'AT76', 'A124', 'CRJ2', 'AT75', 'A319',
       'B763', 'B38M', 'B734', 'B752', 'A20N', 'B735', 'E170', 'B762',
       'BCS3', 'A21N', 'A321', 'B733', 'DC87', 'B789', 'A388', 'B77L',
       'E145', 'PC12', 'AT72', 'B748', 'B744'], dtype=object)

##### Analysis of results

In [ ]:
# plot_data.loc[0,"country_name"] = 'UAE'
# plot_data.loc[12,"country_name"] = 'Tanzania'
# plot_data.loc[17,"country_name"] = 'Korea'
# plot_data.loc[20,"country_name"] = 'Congo'
# plot_data.loc[21,"country_name"] = 'Russia'
sorted_indices = sorted(range(len(plot_data['total_delegates'])), key=lambda i: plot_data['total_delegates'][i])
sorted_categories = [plot_data['country_name'][i] for i in sorted_indices]
sorted_values = [plot_data['total_delegates'][i] for i in sorted_indices]

fig = go.Figure(data=[go.Bar(
    y=sorted_categories,  # y-axis categories
    x=sorted_values,      # x-axis values
    orientation='h'  # horizontal orientation
)])

# Add title and labels
fig.update_layout(
    title='Participants from each country (Top 40)',
    yaxis=dict(title='Countries'),
    xaxis=dict(title='# participants'),
    width=400,
    height=800
)

# Show the chart
#fig.write_image("participants_country.png")
fig.show()
#pio.write_image(fig, "participants_country.png")

## Old 
#### First_closest // Single iteration with further split

In [ ]:
merged_df = pd.merge(df_stopover, df, left_on=['icao', 'closest'], right_on=['origin_airport', 'destination_airport'], how='inner')
merged_df = merged_df.drop_duplicates(subset=['icao'], keep='first')
merged_df.reset_index(inplace=True)
merged_df = merged_df.rename(columns={'destination_airport':'hub_airport'})
merged_df = merged_df[['origin_airport','hub_airport',
       'assumed_load_factor', 'aircraft_type_icao', 'total_fuel_burn','co2_kg', 
       'nox_kg','hc_kg', 'oc_kg', 'h2o_kg', 'so2_kg', 'sulphates_kg','to_dubai'
       ]]
print(len(merged_df))
merged_df.iloc[:3]

0


,origin_airport,hub_airport,assumed_load_factor,aircraft_type_icao,total_fuel_burn,co2_kg,nox_kg,hc_kg,oc_kg,h2o_kg,so2_kg,sulphates_kg,to_dubai


##### Hub -> Dubai (First_closest)

In [ ]:
# Remaining flights to be found! 
occurrences_count = merged_df['origin_airport'].value_counts().to_dict()

# Delete the same number of occurrences from df_stopover['icao'] for each value in final_merged_df['origin_airport']
for value, count in occurrences_count.items():
    df_stopover = df_stopover.drop(df_stopover[df_stopover['icao'] == value].head(count).index)

len(df_stopover)

30

In [ ]:
results_closest = merged_df[['hub_airport','to_dubai']]
destination_dubai = pd.merge(results_closest, df, left_on=['hub_airport', 'to_dubai'], right_on=['origin_airport', 'destination_airport'], how='inner')
destination_dubai = destination_dubai.drop_duplicates(subset=['hub_airport'], keep='first')
destination_dubai.reset_index(inplace=True)
# destination_dubai = destination_dubai[['icao','first_closest','origin_airport','destination_airport',
#        'assumed_load_factor', 'aircraft_type_icao', 'total_fuel_burn','co2_kg', 
#        'nox_kg','hc_kg', 'oc_kg', 'h2o_kg', 'so2_kg', 'sulphates_kg','to_dubai'
#        ]]
print(len(destination_dubai))
destination_dubai = destination_dubai[['origin_airport','hub_airport',
       'assumed_load_factor', 'aircraft_type_icao', 'total_fuel_burn','co2_kg', 
       'nox_kg','hc_kg', 'oc_kg', 'h2o_kg', 'so2_kg', 'sulphates_kg','to_dubai'
       ]]
destination_dubai.iloc[:3]
# destination_dubai['hub_airport'].unique()
# destination_dubai['aircraft_type_icao'].unique()

0


,origin_airport,hub_airport,assumed_load_factor,aircraft_type_icao,total_fuel_burn,co2_kg,nox_kg,hc_kg,oc_kg,h2o_kg,so2_kg,sulphates_kg,to_dubai


In [ ]:
combined_dfs = []
# Iterate over unique hub_airport values
for hub_airport in merged_df['hub_airport'].unique():
    # Filter the row from destination_dubai
    row = destination_dubai[destination_dubai['origin_airport'] == hub_airport].iloc[0]

    # Number of times to add the row
    num_columns_to_add = (merged_df['hub_airport'] == hub_airport).sum()

    # Create a list of copies of row
    copies_of_row = [row] * num_columns_to_add

    # Concatenate the copies along the columns axis
    combined_df = pd.concat(copies_of_row, axis=1)

    # Transpose the DataFrame to have the rows and columns swapped
    combined_df = combined_df.T

    # Append the combined_df to the list
    combined_dfs.append(combined_df)

all_flights_non_direct = pd.concat([merged_df] + combined_dfs, ignore_index=True)
print(len(all_flights_non_direct))
all_flights_non_direct.iloc[:3]

0


,origin_airport,hub_airport,assumed_load_factor,aircraft_type_icao,total_fuel_burn,co2_kg,nox_kg,hc_kg,oc_kg,h2o_kg,so2_kg,sulphates_kg,to_dubai


## Heatmap

In [73]:
scatter_geo = go.Scattergeo(
    locationmode='USA-states',  # Set location mode
    lon=df_stopover['longitude'],
    lat=df_stopover['latitude'],
    # text=df_stopover[''],
    mode='markers',
    marker=dict(
        size=df_stopover['total_delegates']/100,
        # #color=df_stopover['total_delegates']/100,
        # colorscale='Viridis',  # Choose a color scale
        # colorbar=dict(title='Values'),  # Add color bar
        line_width=0.5
    )
)

# Create the layout for the figure
layout = go.Layout(
    title='Global Heatmap with Locations and Size Factor',
    geo=dict(
        scope='world',
        showland=True,
        landcolor='rgb(217, 217, 217)',  # Set the color of land areas
        showframe=False,  # Hide the frame
        projection_type='mercator'  # Choose projection type
    ),
    height=800,  # Increase the height of the plot
)

# Create the figure object
fig = go.Figure(data=[scatter_geo], layout=layout)

# Show the figure
fig.show()

In [74]:
# Sample data
locations = df_stopover['country_code']
latitudes = df_stopover['latitude']
longitudes = df_stopover['longitude']
sizes = df_stopover['total_delegates']/10
#values = [20, 30, 25, 40]  # Values for the color scale

# Initialize the map centered around the first location
mymap = folium.Map()

# Iterate over locations
for lat, lon, size, location in zip(latitudes, longitudes, sizes, locations):
    # Add circle marker with varying sizes
    folium.CircleMarker(
        location=[lat, lon],
        radius=size / 10,  # Normalize size for better visualization
        color='blue',
        fill=True,
        fill_color='blue',
        fill_opacity=0.6,
        popup=location
    ).add_to(mymap)

# Save the map to an HTML file
mymap.save("origin_participants_remainings.html")

In [ ]:
# basemap = folium.Map()
# FastMarkerCluster(airport_location[['latitude', 'longitude']]).add_to(basemap)
# HeatMap(airport_location[['latitude', 'longitude','total_delegates']],
#         ).add_to(basemap)
# basemap

basemap = folium.Map()
FastMarkerCluster(airport_location[['latitude', 'longitude']]).add_to(basemap)
HeatMap(airport_location[['latitude', 'longitude','total_delegates']],
        max_val=max(airport_location['total_delegates'])).add_to(basemap)
basemap

C:\Users\lukas\AppData\Local\Temp\ipykernel_14380\2891524790.py:9: UserWarning:

The `max_val` parameter is no longer necessary. The largest intensity is calculated automatically.

